# 3. FastAPI CRUD
In this notebook we evolve the FastAPI app from the previous lesson into a full CRUD service. You'll build endpoints to create, read, update, and delete tasks, then add a small search feature.

### What you'll practice
- Model request/response bodies with Pydantic
- Wire up CRUD endpoints using the appropriate HTTP verbs
- Use `TestClient` inside the notebook to exercise the API
- Extend the API with a small search enhancement

Run cells in order so the in-memory data stays consistent.


### Quick glossary
- **CRUD**: Create, Read, Update, Delete — the four basic operations an API often supports for a resource.
- **Endpoint**: A specific URL + HTTP method combination (e.g., `GET /tasks/`) that does one job.
- **Pydantic**: Library FastAPI uses to validate/serialize data using Python type hints.
- **BaseModel**: Pydantic base class; you subclass it to define the shape of request/response bodies.
- **FastAPI dependency injection**: FastAPI auto-creates and injects your model instances and path/query params from the HTTP request.
- **response_model**: FastAPI option that declares the type/shape returned to clients; it also filters fields.
- **status_code**: Explicit HTTP status returned (e.g., `201` for created, `204` for no content).
- **HTTPException**: Helper to return an HTTP error with status and message instead of raising a generic Python error.
- **TestClient**: Utility (from `fastapi.testclient`) that lets you call your API in tests without running a server.


In [1]:
# Imports
from fastapi import FastAPI, HTTPException
from fastapi.testclient import TestClient
from pydantic import BaseModel
from typing import List, Optional

## 3.1 Pydantic Models
Pydantic is a library for data validation using Python type hints. FastAPI uses it to define the "shape" of the data we expect to receive or send.

In [2]:
# Define a Pydantic model for a Task
class Task(BaseModel):
    id: Optional[int] = None
    title: str
    description: Optional[str] = None
    completed: bool = False

# Create the app
app = FastAPI()
client = TestClient(app)

# In-memory database
tasks_db = []

## 3.2 Create (POST)
We use the `POST` method to create new resources.
FastAPI will automatically validate the request body against the `Task` model.

In [3]:
@app.post("/tasks/", response_model=Task, status_code=201)
def create_task(task: Task):
    # Generate an ID (simple auto-increment simulation)
    task.id = len(tasks_db) + 1
    tasks_db.append(task)
    return task

# Test creating a task
new_task_data = {"title": "Learn FastAPI", "description": "It is cool"}
response = client.post("/tasks/", json=new_task_data)
print(response.status_code)
print(response.json())

201
{'id': 1, 'title': 'Learn FastAPI', 'description': 'It is cool', 'completed': False}


## 3.3 Read (GET)
We need endpoints to get all tasks and to get a specific task by ID.

In [4]:
@app.get("/tasks/", response_model=List[Task])
def read_tasks():
    return tasks_db

@app.get("/tasks/{task_id}", response_model=Task)
def read_task(task_id: int):
    for task in tasks_db:
        if task.id == task_id:
            return task
    raise HTTPException(status_code=404, detail="Task not found")

# Test reading all tasks
print("All tasks:", client.get("/tasks/").json())

# Test reading one task
print("Task 1:", client.get("/tasks/1").json())

# Test reading non-existent task
print("Task 99:", client.get("/tasks/99").json())

All tasks: [{'id': 1, 'title': 'Learn FastAPI', 'description': 'It is cool', 'completed': False}]
Task 1: {'id': 1, 'title': 'Learn FastAPI', 'description': 'It is cool', 'completed': False}
Task 99: {'detail': 'Task not found'}


## 3.4 Update (PUT)
We use `PUT` to update an existing resource.

In [5]:
@app.put("/tasks/{task_id}", response_model=Task)
def update_task(task_id: int, updated_task: Task):
    for i, task in enumerate(tasks_db):
        if task.id == task_id:
            # Update the fields
            updated_task.id = task_id # Ensure ID stays the same
            tasks_db[i] = updated_task
            return updated_task
    raise HTTPException(status_code=404, detail="Task not found")

# Test updating a task
update_data = {"title": "Learn FastAPI", "description": "It is SUPER cool", "completed": True}
response = client.put("/tasks/1", json=update_data)
print("Updated Task:", response.json())

Updated Task: {'id': 1, 'title': 'Learn FastAPI', 'description': 'It is SUPER cool', 'completed': True}


## 3.5 Delete (DELETE)
We use `DELETE` to remove a resource.

In [6]:
@app.delete("/tasks/{task_id}", status_code=204)
def delete_task(task_id: int):
    for i, task in enumerate(tasks_db):
        if task.id == task_id:
            tasks_db.pop(i)
            return
    raise HTTPException(status_code=404, detail="Task not found")

# Test deleting a task
response = client.delete("/tasks/1")
print("Delete Status:", response.status_code)

# Verify it's gone
print("All tasks after delete:", client.get("/tasks/").json())

Delete Status: 204
All tasks after delete: []


## 3.6 Exercise: Search Feature
**Task**:
1. Modify the `read_tasks` endpoint (or create a new one like `/search/`) to accept a query parameter `keyword`.
2. If `keyword` is provided, return only tasks where the `keyword` is in the `title`.
3. If no `keyword` is provided, return all tasks.

### How to approach the search exercise
- Start with the existing `read_tasks` handler; reuse its return shape (`List[Task]`).
- Accept `keyword` as an optional query parameter and handle the `None` case first.
- Normalize text with `.lower()` for case-insensitive matches.
- Keep mutation out of this handler—just filter the current `tasks_db`.
- Add a couple of `client.get` calls to validate both filtered and unfiltered responses.


In [7]:
# TODO: Write your code here

### Solution

In [8]:
# Solution
# Note: In FastAPI, you can't easily "overwrite" an endpoint in the same app instance in a notebook cell 
# without redefining the app or using a different path. 
# For this exercise, let's create a specific search endpoint.

@app.get("/search/", response_model=List[Task])
def search_tasks(keyword: Optional[str] = None):
    if keyword:
        return [task for task in tasks_db if keyword.lower() in task.title.lower()]
    return tasks_db

# Add some dummy data to test
client.post("/tasks/", json={"title": "Buy milk"})
client.post("/tasks/", json={"title": "Buy eggs"})
client.post("/tasks/", json={"title": "Walk the dog"})

# Test search
print("Search 'Buy':", client.get("/search/?keyword=Buy").json())
print("Search 'dog':", client.get("/search/?keyword=dog").json())

Search 'Buy': [{'id': 1, 'title': 'Buy milk', 'description': None, 'completed': False}, {'id': 2, 'title': 'Buy eggs', 'description': None, 'completed': False}]
Search 'dog': [{'id': 3, 'title': 'Walk the dog', 'description': None, 'completed': False}]
